# Converting script to generate from the csv the pytorch dataset.

### this script convert the csv data provided by meteotrentino in pytorch dataset usable by the LSTM. This do not convert the complex-LSTM (multiple features, like wind direction). It can work in two configuration: getting the dataset by googledrive (use google colab), or on local data, downloaded. The csv data is readed, splittend in PERIOD-h with PREDICTION-h a

In [1]:
# load required libraries
import pandas as pd
import os
import numpy as np
import datetime
import traceback
from torch.utils.data import Dataset, DataLoader
from torch import save,load

In [2]:
# mount drive
drive.mount('/content/drive')


In [13]:
# Warning: location dependent!
#                if local folders: --------
# point to the Meteotrentino drive folder
wk =  "/home/pierobon/Documents/data/exp/meteo_TN/raw/"

# folder fot the pytorch
pt_subfolder = "DatasetPytorch/"

# change me, have to point the dataset to convert
csv_name = "temperatura3_A.csv"
data_path = wk+'csv_data_from_gDrive/'+csv_name 

station_path = wk + "lista stazioni con sensori_aggiornato al 06072022.xlsx"


#                if drive folders: ---------
## point to the Meteotrentino drive folder
#wk =  "/content/drive/My Drive/10. Meteotrentino/"
#
## folder for the csv
#csv_subfolder = "Dati csv convalidati/"
#
## folder fot the pytorch
pt_subfolder = wk+"pytorch_dataset/"


## change me, have to point the dataset to convert
#data_path = wk+csv_subfolder+"umidità_A.csv"
#
#station_path = wk + "Informazioni sulle stazioni/lista stazioni con sensori_aggiornato al 06072022.xlsx"



# in both cases: --------
# conversion log path
log_folder = wk+"conversion_logs/"



In [4]:
# select the filename of the dataset (Dario: dont judge me)
# I don't know how it works
selected_dataset = os.path.basename(data_path).split('_A')[0] 
if selected_dataset > "vel":
  dataset_sensor = "vento"
elif selected_dataset > "umidita":
  dataset_sensor = "umidità"
elif selected_dataset > "temperatura":    
  dataset_sensor = "temperatura"
elif selected_dataset > "radiazione":
  dataset_sensor = "radiazione"
elif selected_dataset > "pioggia":
  dataset_sensor = "pioggia"
elif selected_dataset > "direzione":
  dataset_sensor = "vento"
else:
  dataset_sensor = "Altezza neve"
print(dataset_sensor)

temperatura


In [5]:
# load the dataset
df_row_data = pd.read_csv(
  data_path,
  sep = ";", # not a CSV, separator is ';'
  header = None, # no header
  dtype = {'0' : str, '1': str, '2': str, '3' : np.float64, '4':int} # Daniele: added the validation code so we can train on validated data (no need of classes, just distinguish between good and bad data)
  ) 

# define a header 
df_row_data = df_row_data.set_axis(
    ['label', 'date', 'time', 'value','validation_code'],
    axis = 1,
    inplace = False #inplace deprecated
)
# create a compact datetime timestamp
df_row_data['datetime'] = pd.to_datetime(df_row_data['date']+" "+df_row_data['time'], format = '%d/%m/%Y %H:%M')
  

In [6]:
# pytorch class to create the dataset. 
import torch
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
  def __init__(self, df_train, df_test, transform = None, target_transform = None):
    self.features = torch.FloatTensor(df_train.drop(['sensor_code','in_datetime'],axis=1).values)
    ### standardize the data
    means = self.features.mean(dim=1, keepdim=True)
    stds = torch.clamp(self.features.std(dim=1, keepdim=True),min=1)
    self.features = (self.features - means) / stds
    self.target = torch.FloatTensor(df_test.drop(['sensor_code','in_datetime']+[f"val_{idx}" for idx in range(n_previsions)],axis=1).values)
    self.target = (self.target - means) / stds
    
    self.sensor_code = df_train['sensor_code'].values
    self.in_datetime = df_train['in_datetime'].values
    
    self.labels = torch.FloatTensor(df_test[[f"val_{idx}" for idx in range(n_previsions)]].values)
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.sensor_code)

  def __getitem__(self, idx):
    sensor_code = self.sensor_code[idx]
    date = self.in_datetime[idx]
    features = self.features[idx]
    labels = self.labels[idx]
    target = self.target[idx]
    return {"sensor_code": sensor_code, "date":date,"features": features, "target":target,"labels":labels}

In [ ]:
from importlib import reload  # Python 3.4+
import sys

# add the path to TSF/data_splitter
!git clone https://github.com/eliazonta/AI-Challenge-HIT

# if drive:
sys.path.append("content/AI-Challenge-HIT/python_code/notebooks/TSF")
                
# if local
sys.path.append("/home/pierobon/Documents/code/python/code/ML/challenge_IA/AI-Challenge-HIT/python_code/notebooks/TSF/")

import data_splitter
data_splitter = reload(data_splitter)
from data_splitter import *
from torch import save,load


In [37]:
# select period and prediction. 
PERIOD = 24
PREDICTION = 1

for station in pd.unique(df_row_data['label']):
    print("Processing station = "+station)
    df_single_station = df_row_data[df_row_data['label'] == station].reset_index(drop=True)
    if not df_single_station.empty: #some stations has the sensor but not the data idk why
        print("Start...")
        df = df_single_station
        print("Splitting...")
        df_train,df_prediction,n_previsions = split_df_for_TSF(
            df, # the dataframe
            PERIOD, #period
            PREDICTION, #prediction
            500, #print modulus
            log_folder+station+"_"+csv_name[:-4]+".log" #file log path.
        )
        try:
            some_object_iterator = iter(df_train)
        except TypeError as te:
            print("Unable to process this station")
            continue
        print("Splitting done")
        
        #df_prediction.to_csv( wk + csv_subfolder + "/" + station + "_" + selected_dataset + "_" + "prediction.csv") #Save it on GDrive
        splits = ['train','test']
        shuffle = {'train':True,'test':False}
        dataset = {}

        print("Resetting index")
        # split data in training and test (to check generalization)
        training_features = df_train.sample(frac=0.8)
        training_target = df_prediction.iloc[training_features.index.values]
        test_features = df_train.drop(training_features.index)
        test_target = df_prediction.iloc[test_features.index]
        
        
        training_features = training_features.reset_index(drop=True)
        training_target = training_target.reset_index(drop=True)
        test_features = test_features.reset_index(drop=True)
        test_target = test_target.reset_index(drop=True)

        print("Creating custom datasets")
        dataset['train'] = CustomDataset(training_features, training_target)
        dataset['test'] = CustomDataset(test_features, test_target)
        
        # save
        save(dataset['train'], pt_subfolder + station + "_train_"+ selected_dataset + "_tsf.pt") 
        save(dataset['test'],  pt_subfolder + station + "_test_"+ selected_dataset + "_tsf.pt")
        print (station + " saved")
    else:
        print("Station was empty")


Processing station = T0431
Start...
Splitting...
Logging to /home/pierobon/Documents/data/exp/meteo_TN/raw/conversion_logs/T0431_temperatura3_A.log
Splitting done
Resetting index
Creating custom datasets
T0431 saved
Processing station = T0432
Start...
Splitting...
Logging to /home/pierobon/Documents/data/exp/meteo_TN/raw/conversion_logs/T0432_temperatura3_A.log
Splitting done
Resetting index
Creating custom datasets
T0432 saved
Processing station = T0433
Start...
Splitting...
Logging to /home/pierobon/Documents/data/exp/meteo_TN/raw/conversion_logs/T0433_temperatura3_A.log
Splitting done
Resetting index
Creating custom datasets
T0433 saved
Processing station = T0435
Start...
Splitting...
Logging to /home/pierobon/Documents/data/exp/meteo_TN/raw/conversion_logs/T0435_temperatura3_A.log
Splitting done
Resetting index
Creating custom datasets
T0435 saved
Processing station = T0437
Start...
Splitting...
Logging to /home/pierobon/Documents/data/exp/meteo_TN/raw/conversion_logs/T0437_tempera